# Complex Network Analysis
## Part 1: Structural Characterization

In [19]:
import networkx as nx
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import numpy as np

NETWORKS = ['net1', 'net2', 'net3', 'net4']
metrics = []

## Macroscopic Analysis

In [20]:
def load_network(net_name):
    """Caricamento ottimizzato per il formato specifico delle reti"""
    path = f'data/{net_name}.net'
    G = nx.Graph()  # Forza grafo non orientato semplice
    
    with open(path, 'r') as f:
        # Legge i vertici saltando le proprietà aggiuntive
        while True:
            line = f.readline()
            if line.startswith('*vertices'):
                n_nodes = int(line.split()[1])
                for _ in range(n_nodes):
                    parts = f.readline().split()
                    node = int(parts[0])
                    G.add_node(node)
                break
                
        # Legge gli archi
        for line in f:
            if line.startswith('*edges'):
                continue
            parts = line.strip().split()
            if len(parts) >= 2:
                u, v = int(parts[0]), int(parts[1])
                G.add_edge(u, v)
    
    return G
    

def analyze_network(G):
    """Analisi adattata per reti ad anello/strutturate"""
    # Calcola metriche di base
    metrics = {
        'Nodes': G.number_of_nodes(),
        'Edges': G.number_of_edges(),
        'Avg Degree': 2 * G.number_of_edges() / G.number_of_nodes()
    }
    
    # Clustering coefficient per reti strutturate
    metrics['Clustering'] = nx.average_clustering(G, count_zeros=True)
    
    # Path analysis su componente connessa
    if nx.is_connected(G):
        metrics['Avg Path Length'] = nx.average_shortest_path_length(G)
        metrics['Diameter'] = nx.diameter(G)
    else:
        giant = max(nx.connected_components(G), key=len)
        metrics['Avg Path Length'] = nx.average_shortest_path_length(G.subgraph(giant))
        metrics['Diameter'] = nx.diameter(G.subgraph(giant))
    
    return metrics

# Batch processing with progress bar
for net in tqdm(NETWORKS, desc='Analyzing networks'):
    G = load_network(net)
    metrics.append(analyze_network(G))

# Create results dataframe
df_metrics = pd.DataFrame(metrics)
df_metrics.to_csv('network_metrics.csv', index=False)

Analyzing networks:   0%|          | 0/4 [00:00<?, ?it/s]

## Microscopic Analysis

In [21]:
def calculate_centralities(G):
    """Calcolo centralità ottimizzato per reti ad anello"""
    # Betweenness approssimato
    betweenness = nx.betweenness_centrality(G, k=min(500, len(G)), seed=42)
    
    # Eigenvector centrality con normalizzazione
    try:
        eigen = nx.eigenvector_centrality(G, max_iter=2000, tol=1e-6)
    except nx.PowerIterationFailedConvergence:
        eigen = {n: 0 for n in G.nodes()}
    
    return {
        'Betweenness': betweenness,
        'Degree': nx.degree_centrality(G),
        'Eigenvector': eigen
    }

centrality_results = {}

for net in NETWORKS:
    G = load_network(net)
    centrality_results[net] = calculate_centralities(G)
    
    # Save top 5 nodes
    for metric, values in centrality_results[net].items():
        top5 = sorted(values.items(), key=lambda x: -x[1])[:5]
        print(f"{net} - {metric}: {[n[0] for n in top5]}")

net1 - Betweenness: [4748, 4725, 1796, 2646, 2547]
net1 - Degree: [1694, 652, 1580, 4892, 42]
net1 - Eigenvector: [652, 1938, 4527, 4399, 1940]
net2 - Betweenness: [4670, 1582, 2813, 2373, 752]
net2 - Degree: [1582, 788, 53, 1991, 2373]
net2 - Eigenvector: [1582, 3234, 788, 2376, 132]
net3 - Betweenness: [8, 6, 1, 3, 5]
net3 - Degree: [6, 8, 3, 1, 7]
net3 - Eigenvector: [6, 8, 3, 1, 4]
net4 - Betweenness: [1, 7, 10, 9, 4]
net4 - Degree: [7, 1, 10, 11, 9]
net4 - Eigenvector: [7, 1, 10, 9, 4]


## Degree Distribution Analysis

In [18]:
import os  # Aggiungi questo import

def plot_degree_distribution(G, net_name):
    # Crea la cartella se non esiste
    os.makedirs('images', exist_ok=True)
    
    degrees = [d for _, d in G.degree()]
    
    plt.figure(figsize=(10, 6))
    if max(degrees) > 1000:
        plt.xscale('log')
        plt.yscale('log')
        bins = np.logspace(np.log10(1), np.log10(max(degrees)), 50)
    else:
        bins = 50
        
    plt.hist(degrees, bins=bins, density=True)
    plt.savefig(f'images/{net_name}_degree_dist.png')
    plt.close()

# Esegui prima la creazione della cartella
os.makedirs('images', exist_ok=True)

for net in NETWORKS:
    G = load_network(net)  # Usa la tua funzione di caricamento modificata
    plot_degree_distribution(G, net)